<a href="https://colab.research.google.com/github/aarondav85/ProyectoTelegram/blob/main/ProyectoTelegram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install selenium
# !pip install python-telegram-bot
import logging
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from selenium import webdriver
from bs4 import  BeautifulSoup

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=options)
ListCodigos = ["JNB","CPT","CAI","ADD","HRG","TFS","CMN","ALG","NBO","LOS",
               "PVG","DXB","HKG","PEK","HND","DMM","BKK","DEL","CAN","ICN",
               "GGK","SIN","LHR","CDG","AMS","FRA","BCN","IST","MAD","MUC",
               "SVO","LGW","ATL","LAX","ORD","DFW","LAS","DEN","JFK","SFO",
               "YYZ","SEA","BOG","CGH","LIM","EZE","MEX","GRU","CUN","SCL",
               "BSB","PTY","GIG","SYD","MEL","BNE","AKL","PER","HNL","ADL",
               "CHC","OOL","OGG"]
ListDestino = ["OR Tambo International Airport, Johannesburg, South Africa","Cape Town International Airport, Cape Town, South Africa",
               "Cairo International Airport, Cairo, Egypt","Addis Ababa Bole International Airport, Addis Ababa, Ethiopia",
               "Hurghada International Airport,	Hurghada,	Egypt","Tenerife South Airport,	Tenerife Island, Spain",
               "Mohammed V International Airport,	Casablanca,	Morocco","Algiers Houari Boumediene Airport, Algiers,	Algeria",
               "Jomo Kenyatta International Airport, Nairobi, Kenya","Murtala Muhammed International Airport, Lagos, Nigeria",
               "Shanghai Pudong International Airport, Shanghai, China","Dubai International Airport, Dubai, United Arab Emirates",
               "Hong Kong International Airport, Hong Kong, Hong Kong","Beijing Capital International Airport, Beijing, China",
               "Tokyo Haneda Airport, Tokyo, Japan","King Fahd International Airport, Ad Dammam, Saudi Arabia",
               "Suvarnabhumi Airport, Bangkok, Thailand","Indira Gandhi International Airport, New Delhi, India",
               "Guangzhou Baiyun International Airport, Guangzhou, China","Seoul Incheon International Airport, Seoul, South Korea",
               "Soekarno-Hatta International Airport, Jakarta,	Indonesia",
               "Singapore Changi Airport, Singapore, Singapore","London Heathrow Airport, London, United Kingdom",
               "Paris-Charles de Gaulle Airport, Paris, France","Amsterdam Airport Schiphol,	Amsterdam, The Netherlands",
               "Frankfurt am Main Airport,	Frankfurt am Main, Germany","Barcelona–El Prat Airport,	Barcelona, Spain",
               "Istanbul Ataturk Airport, Istanbul,	Turkey","Adolfo Suárez Madrid–Barajas Airport, Madrid,	Spain",
               "Munich Airport, Munich,	Germany","Sheremetyevo International Airport, Moscow,	Russia",
               "London Gatwick Airport, London,	United Kingdom","Hartsfield-Jackson Atlanta International Airport, Atlanta, United States",
               "Los Angeles International Airport,	Los Angeles, United States","Chicago O'Hare International Airport, Chicago,	United States",
               "Dallas Fort Worth International Airport,	Dallas-Fort Worth, United States","McCarran International Airport, Las Vegas,	United States",
               "Denver International Airport, Denver, United States","John F. Kennedy International Airport,	New York,	United States",
               "San Francisco International Airport,	San Francisco, United States","Toronto Pearson International Airport,	Toronto, Canada",
               "Seattle-Tacoma International Airport, Seattle,	United States",
               "El Dorado International Airport	Bogota	Colombia","São Paulo–Congonhas Airport,	São Paulo, Brazil",
               "Jorge Chávez International Airport, Lima, Peru","Ministro Pistarini International Airport, Buenos Aires,	Argentina",
               "Mexico City International Airport,	Mexico City, Mexico","São Paulo–Guarulhos International Airport,	São Paulo, Brazil",
               "Cancún International Airport, Cancún, Mexico","Comodoro Arturo Merino Benítez International Airport, Santiago, Chile",
               "Brasília International Airport, Brasília, Brazil","Tocumen International Airport,	Tocumen, Panama",
               "Galeão International Airport, Rio De Janeiro, Brazil",
               "Sydney Airport, Sydney, Australia","Melbourne International Airport, Melbourne, Australia",
               "Brisbane International Airport, Brisbane, Australia","Auckland International Airport, Auckland, New Zealand",
               "Perth International Airport,	Perth, Australia","Daniel K. Inouye International Airport, Honolulu, United States",
               "Adelaide International Airport, Adelaide, Australia","Christchurch International Airport, Christchurch, New Zealand",
               "Gold Coast Airport, Gold Coast, Australia","Kahului Airport,	Kahului, United States"]  
listI = []
listV = []
listAir = []
listPrice = []
listCode = []
def scrap(miCodigo, url):
  #url = "https://www.despegar.com.ec/vuelos/GYE/" + miCodigo + "?from=SB&di=1-0&reSearch=true"
  wd.get(url)
  soup = BeautifulSoup(wd.page_source,'html.parser')
  code = soup.find("div", {"class":"clusters"})
  listI[:] = []
  listV[:] = []
  listAir[:] = []
  listPrice[:] = []
  listCode[:] = []
  if code is None or code==-1:
    txtAux = ""
    for indiceC in range(0, len(ListCodigos)):
      if ListCodigos[indiceC] == miCodigo:
        txtAux = "No hay vuelos disponibles a " + ListDestino[indiceC]
        print("No hay vuelos disponibles a " + ListDestino[indiceC])
        break
    return txtAux
  else:
    datos = code.find_all('div', class_='clusters')
    if datos == -1:
      return 0
    airline = code.find_all('img')
    test = code.findAll('div', class_='eva-3-cluster-basic cluster-clickable')
    for t in test:
      aux = t.find_all('div', class_='date -eva-3-bold route-info-item-date lowercase')
      listI.append("Fecha de ida:" + aux[0].text)
      if len(aux) == 2:
        listV.append("Fecha de vuelta:" + aux[1].text)
      listCode.append(miCodigo)
      auxAir = t.find_all('img')
      txtAir = ""
      for a in auxAir:
        if a.get('alt') is not None:
          txtAir = txtAir + a.get('alt') + ", "
      txtAir = txtAir[:len(txtAir) - 2]
      listAir.append(txtAir)
      auxPrice = t.find('span', class_='pricebox-big-text price')
      listPrice.append(auxPrice.text)
    wd.delete_all_cookies
  return 1

def buscarIndice(buscar):
  for i in range(0, len(ListCodigos)):
    if len(buscar) == 3:
      if buscar.upper() == ListCodigos[i]:
        return i
    else:
      txtAux = ListDestino[i].upper()
      if txtAux.find(buscar.upper()) != -1:
        return i
  return "false"

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)
mensajeEvento = "0"
miDestino = "0"
def start(update, context):
    """Send a message when the command /start is issued."""
    update.message.reply_text("Gestion de vuelos. \n/LIST Para listar vuelos disponibles desde Guayaquil. \n/SEARCHD Para buscar vuelos al destino ingresado \n/BUY_TICKET Para reservar un vuelo solo de ida \n/BUYRT_TICKET Para reservar un vuelo de ida y vuelta")

def help_command(update, context):
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')

def listar_vuelos(update, context):
    update.message.reply_text("Destinos disponibles: ")
    for v in ListDestino:
      update.message.reply_text(v)

mensajeEvento = "nada"
def searchD(update, context):
    global mensajeEvento
    mensajeEvento = "searchD"
    update.message.reply_text("Ingrese el destino (Codigo IATA, ciudad, aeropuerto o país):")
    #update.message.reply_text(mensajeEvento)

def busquedaD(update, context):
    global mensajeEvento
    global miDestino
    #update.message.reply_text(mensajeEvento)
    if mensajeEvento == "searchD":
      mensajeEvento = "busquedaD"
      buscar = update.message.text
      #id = ListCodigos[buscarIndice(buscar)]
      auxI = buscarIndice(buscar)
      if auxI == "false":
        update.message.reply_text("Destino no encontrado")
        return
      else:
        mensajeEvento = "0"
        id = ListCodigos[auxI]
        url = "https://www.despegar.com.ec/vuelos/GYE/" + id + "?from=SB&di=1-0&reSearch=true"
        auxValue = scrap(id, url)
        if auxValue == 1: 
          update.message.reply_text("Vuelos disponibles de IDA Y VUELTA desde Guayaquil a " + ListDestino[auxI] + ":")
          for i in range(0, len(listAir)):
            update.message.reply_text("Aerolineas: " + listAir[i] + "\n" + listI[i] + "\n" + listV[i] + "\n" + "Precio: " + listPrice[i] + "$")
        else:
          update.message.reply_text(auxValue)
        return
    if mensajeEvento == "BUY_TICKET":
      miDestino = update.message.text
      mensajeEvento = "Asiento01"
      update.message.reply_text("Ingrese el numero de asientos: ")
      return
    if mensajeEvento == "Asiento01":
      asientos = int(update.message.text)
      #####
      auxI = buscarIndice(miDestino)
      if auxI == "false":
        update.message.reply_text("Destino no encontrado")
        return
      else:
        mensajeEvento = "0"
        id = ListCodigos[auxI]
        url = "https://www.despegar.com.ec/vuelos/GYE/" + id + "?from=SB&di=1-0&searchType=ONEWAY&reSearch=true#showModal"
        auxValue = scrap(id, url)
        if auxValue == 1: 
          update.message.reply_text("Vuelos disponibles de SOLO IDA desde Guayaquil a " + ListDestino[auxI] + ":")
          for i in range(0, len(listAir)):
            tempV = int(listPrice[i].replace('.','')) * asientos
            update.message.reply_text("Aerolineas: " + listAir[i] + "\n" + listI[i] + "\n" + "Precio: " + str(tempV) + "$" + "\nAsientos: " + str(asientos))
        else:
          update.message.reply_text(auxValue)
        return
      ####
    if mensajeEvento == "BUYRT_TICKET":
      miDestino = update.message.text
      mensajeEvento = "Asiento02"
      update.message.reply_text("Ingrese el numero de asientos: ")
      return
    if mensajeEvento == "Asiento02":
      asientos = int(update.message.text)
      auxI = buscarIndice(miDestino)
      if auxI == "false":
        update.message.reply_text("Destino no encontrado")
        return
      else:
        mensajeEvento = "0"
        id = ListCodigos[auxI]
        url = "https://www.despegar.com.ec/vuelos/GYE/" + id + "?from=SB&di=1-0&reSearch=true"
        auxValue = scrap(id, url)
        if auxValue == 1: 
          update.message.reply_text("Vuelos disponibles de IDA Y VUELTA desde Guayaquil a " + ListDestino[auxI] + ":")
          for i in range(0, len(listAir)):
            tempV = int(listPrice[i].replace('.','')) * asientos
            update.message.reply_text("Aerolineas: " + listAir[i] + "\n" + listI[i] + "\n" + listV[i] + "Precio: " + str(tempV) + "$" + "\nAsientos: " + str(asientos))
        else:
          update.message.reply_text(auxValue)
        return

def BUYRT_TICKET(update, context):
    global mensajeEvento
    mensajeEvento = "BUYRT_TICKET"
    update.message.reply_text("Ingrese el destino (Codigo IATA, ciudad, aeropuerto o país):")

def BUY_TICKET(update, context):
    global mensajeEvento
    mensajeEvento = "BUY_TICKET"
    update.message.reply_text("Ingrese el destino (Codigo IATA, ciudad, aeropuerto o país):")


def main():
    updater = Updater("1158169178:AAGBfOwD7l1PwhrVZOkev8J7hAkpds4alxk", use_context=True)
    dp = updater.dispatcher
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("help", help_command))
    dp.add_handler(CommandHandler("LIST", listar_vuelos))
    dp.add_handler(CommandHandler("SEARCHD", searchD))
    dp.add_handler(CommandHandler("BUY_TICKET", BUY_TICKET))
    dp.add_handler(CommandHandler("BUYRT_TICKET", BUYRT_TICKET))
    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, busquedaD))
    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()

2020-10-14 05:27:36,949 - apscheduler.scheduler - INFO - Scheduler started
2020-10-14 05:55:11,160 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2020-10-14 05:55:11,164 - apscheduler.scheduler - INFO - Scheduler has been shut down
